In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
  
from tensorflow.keras import models, layers, utils 
from sklearn import metrics, preprocessing
import os
import gc
import sys

import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
sns.set_style("darkgrid")

gc.collect()


202

## recommender system 
the two main functions are 
get_similar_movies(movie_title, min_ratings_count = 0)

recommend_movies(user_id , n_movies= 10)


In [ ]:
path = "D:\\netflix dataset\\netflix-prize-data"


In [ ]:
start = datetime.now()
if not os.path.isfile('data.csv'):
    data = open('data.csv', mode='w')
    row = list()
    files = os.listdir(path)
    print(files)
    for file in files:
        if file.startswith('combined_data'):
        # if file.startswith('combined_data_1'):   # use this line to process only one file
            print("Reading ratings from {}\n".format(file))
            with open(os.path.join(path,file)) as f:
                for line in f:
                    line = line.strip()
                    if line.endswith(':'):
                        # All below are ratings for this movie, until another movie appears.
                        movie_id = line.replace(':', '')
                    else:
                        row = [x for x in line.split(',')]
                        row.insert(0, movie_id)
                        data.write(','.join(row))
                        data.write('\n')
data.close()
print('Time taken :', datetime.now() - start)


In [12]:
df = pd.read_csv('data.csv', names=['Movie_Id', 'Cust_Id', 'Rating', 'Date'])


In [13]:
df.head()


,Movie_Id,Cust_Id,Rating,Date
0,1,1488844,3,2005-09-06
1,1,822109,5,2005-05-13
2,1,885013,4,2005-10-19
3,1,30878,4,2005-12-26
4,1,823519,3,2004-05-03


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 4 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  int64 
 1   Cust_Id   int64 
 2   Rating    int64 
 3   Date      object
dtypes: int64(3), object(1)
memory usage: 3.0+ GB


In [15]:
df['Movie_Id'] = df['Movie_Id'].astype(np.uint32)
df['Cust_Id'] = df['Cust_Id'].astype(np.uint32)
df['Rating'] = df['Rating'].astype(np.uint8)
df.drop(['Date'], axis=1, inplace=True)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100480507 entries, 0 to 100480506
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 862.4 MB


In [16]:
f = ['count', 'mean']

df_movie_summary = df.groupby('Movie_Id')['Rating'].agg(f)
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(0.7), 0)
drop_movie_list = df_movie_summary[df_movie_summary['count']
                                   < movie_benchmark].index
print(len(drop_movie_list))
print('Movie minimum times of review: {}'.format(movie_benchmark))

### i decided not to drop any customer data since later when we will recommend movies to a customer, we will need to know the customer's rating history

df_cust_summary = df.groupby('Cust_Id')['Rating'].agg(f)
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(0.7), 0)
drop_cust_list = df_cust_summary[df_cust_summary['count']
                                 < cust_benchmark].index

print('Customer minimum times of review: {}'.format(cust_benchmark))

print('Original Shape: {}'.format(df.shape))
df = df[~df['Movie_Id'].isin(drop_movie_list)]
# df.drop(df['Movie_Id'].isin(drop_movie_list), axis=0, inplace=True)
df = df[~df['Cust_Id'].isin(drop_cust_list)]
print('After Trim Shape: {}'.format(df.shape))


print("number of movies deleted is " , len(drop_movie_list))
# print("number of customers deleted is " , len(drop_cust_list))

12438
Movie minimum times of review: 1948.0
Customer minimum times of review: 211.0
Original Shape: (100480507, 3)
After Trim Shape: (71833509, 3)
number of movies deleted is  12438


In [17]:
print(len(drop_movie_list))


12438


In [18]:
# del df_cust_summary, df_movie_summary, drop_movie_list, drop_cust_list
# gc.collect()
print(df.info())
df.index = range(len(df))

print(df.info())



<class 'pandas.core.frame.DataFrame'>
Int64Index: 71833509 entries, 693 to 100479585
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 1.1 GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71833509 entries, 0 to 71833508
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Movie_Id  uint32
 1   Cust_Id   uint32
 2   Rating    uint8 
dtypes: uint32(2), uint8(1)
memory usage: 616.6 MB
None


In [21]:
df_title = pd.read_csv("D:\\netflix dataset\\netflix-prize-data\\movie_titles.csv", encoding="ISO-8859-1",
                        header=None, names=['Movie_Id', 'Year', 'Name'], usecols=[0, 1, 2])
df_title.set_index('Movie_Id', inplace=True)
df_title['Year'].fillna(0, inplace=True)
df_title['Year'] = df_title['Year'].astype(int)
df_title.head(10)


,Year,Name
Movie_Id,,
1,2003,Dinosaur Planet
2,2004,Isle of Man TT 2004 Review
3,1997,Character
4,1994,Paula Abdul's Get Up & Dance
5,2004,The Rise and Fall of ECW
6,1997,Sick
7,1992,8 Man
8,2004,What the #$*! Do We Know!?
9,1991,Class of Nuke 'Em High 2


In [19]:
df_pivot = pd.pivot_table(df, values='Rating',
                      index='Cust_Id', columns='Movie_Id')

print(df_pivot.shape)


(144380, 5332)


In [20]:
df_pivot.info()
# nearly 6 gb of ram is used by pivot table

<class 'pandas.core.frame.DataFrame'>
UInt64Index: 144380 entries, 6 to 2649429
Columns: 5332 entries, 3 to 17769
dtypes: float64(5332)
memory usage: 5.7 GB


In [22]:
# min count is min count of ratings given by a customer
def get_similar_movies(movie_title, min_ratings_count=0):
    print("For movie ({})".format(movie_title))
    print("- Top 10 movies recommended based on Pearsons'R correlation - ")
    i = int(df_title.index[df_title['Name'] == movie_title][0])
    target = df_pivot[i]
    similar_to_target = df_pivot.corrwith(target)
    corr_target = pd.DataFrame(similar_to_target, columns=['PearsonR'])
    corr_target.dropna(inplace = True)
    corr_target = corr_target.sort_values('PearsonR', ascending = False)
    corr_target.index = corr_target.index.map(int)
    corr_target = corr_target.join(df_title).join(df_movie_summary)[['PearsonR', 'Name', 'count', 'mean']]
    corr_target.rename(columns={"Name": "movie name", "count": "number of ratings"  , "mean": "average rating" ,
                                "PearsonR": "Pearson correlation coefficient"}, inplace=True)
    
    print(corr_target[corr_target['number of ratings']>min_ratings_count][1:11].to_string(index=False))

In [23]:
get_similar_movies("Lord of the Rings: The Fellowship of the Ring", 30000)
# you can see it outputs similar movies 


For movie (Lord of the Rings: The Fellowship of the Ring)
- Top 10 movies recommended based on Pearsons'R correlation - 
 Pearson correlation coefficient                                                          movie name  number of ratings  average rating
                        0.809049                                   Lord of the Rings: The Two Towers             151245        4.460676
                        0.756614                           Lord of the Rings: The Return of the King             134284        4.545121
                        0.691485 The Lord of the Rings: The Fellowship of the Ring: Extended Edition              73422        4.716611
                        0.657337                 Lord of the Rings: The Two Towers: Extended Edition              74912        4.702611
                        0.631096         Lord of the Rings: The Return of the King: Extended Edition              73335        4.723270
                        0.259167                               

In [24]:
get_similar_movies("Fight Club")
get_similar_movies("Fight Club", 80000)
# here it actually ouputs movies of the same genre or the the same director
# and min count produces better recommendations if used correctly

For movie (Fight Club)
- Top 10 movies recommended based on Pearsons'R correlation - 
 Pearson correlation coefficient                            movie name  number of ratings  average rating
                        0.380089                            12 Monkeys              76475        3.776084
                        0.378151                                Snatch              68818        3.848005
                        0.364983                               Memento             127318        3.851427
                        0.357912                          Pulp Fiction             152078        4.094235
                        0.347596                         Trainspotting              24446        3.885953
                        0.343847 Eternal Sunshine of the Spotless Mind             110159        3.695304
                        0.333834                        Reservoir Dogs              90450        4.003847
                        0.326851  The Royal Tenenbaums: Bonus Mate

In [25]:
get_similar_movies("Saving Private Ryan", 0)
# here it ouputs similar war movies


For movie (Saving Private Ryan)
- Top 10 movies recommended based on Pearsons'R correlation - 
 Pearson correlation coefficient        movie name  number of ratings  average rating
                        0.348649   Black Hawk Down              77597        3.859608
                        0.343554        Braveheart             135601        4.294423
                        0.328930         Apollo 13              98116        4.082576
                        0.321378         Gladiator             150592        4.202713
                        0.321019      Forrest Gump             181508        4.299910
                        0.315738  Band of Brothers              37514        4.513328
                        0.315564           Platoon              64175        3.986038
                        0.303892  We Were Soldiers              64917        4.005299
                        0.295405  Schindler's List             101141        4.458004
                        0.292609 Sands of Iwo

In [26]:
del df_pivot
gc.collect()

30

### collaborative filtering
Singular Value Decomposition (SVD)

In [40]:
# first we can observe like movies of a customer
def get_prev_liked_movies(user_id, num_movies=10 , verbose=False):
    # liked_movies = df[(df['Cust_Id'] == user_id) & (df['Rating'] == rating)]
    liked_movies = df[(df['Cust_Id'] == user_id)]
    liked_movies.set_index('Movie_Id', inplace=True)
    liked_movies = liked_movies.join(df_title)[['Name', 'Year', 'Rating']]
    # print(liked_movies.unique())
    if verbose:
        print("Liked movies by user {} are : ".format(user_id))
        print(liked_movies.head(num_movies).to_string(index=False))
    # return the id of liked movies
    return liked_movies.index

get_prev_liked_movies(1987434 , verbose=True)


Liked movies by user 1987434 are : 
                          Name  Year  Rating
    What the #$*! Do We Know!?  2004       4
               Lilo and Stitch  2002       4
        Something's Gotta Give  2003       2
Aqua Teen Hunger Force: Vol. 1  2000       4
                   Richard III  1995       4
                   Dragonheart  1996       3
            Jingle All the Way  1996       1
                       Spartan  2004       2
                  Fatal Beauty  1987       2
         Viva La Bam: Season 1  2003       4


UInt64Index([    8,    28,    30,    33,    57,    58,    78,   108,   127,
               133,
             ...
             17586, 17603, 17621, 17622, 17625, 17627, 17692, 17703, 17714,
             17764],
            dtype='uint64', name='Movie_Id', length=1076)

In [41]:
len (get_prev_liked_movies(1987434, verbose= False))


Liked movies by user 1987434 are : 


1076

In [61]:


reader = Reader()

# get just top 100K rows for faster run time
data = Dataset.load_from_df(
    df[['Cust_Id', 'Movie_Id', 'Rating']][:10000000], reader)

# data.split(n_folds=3)
# trainset = data.build_full_trainset()

# param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# grid_search = GridSearchCV(SVD, param_grid, measures=['RMSE', 'FCP'], cv=3)
# grid_search.fit(data)
# print(grid_search.best_score['RMSE'])
# print(grid_search.best_params['RMSE'])
# print(grid_search.best_score['FCP'])
# print(grid_search.best_params['FCP'])
# print(grid_search.best_score)


svd = SVD()
cross_validate(svd, data, measures=['RMSE'] , cv=5, verbose=True)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8655  0.8665  0.8658  0.8672  0.8666  0.8663  0.0006  
Fit time          69.25   69.40   69.03   65.18   65.34   67.64   1.95    
Test time         23.46   23.96   21.79   22.29   22.25   22.75   0.82    


{'test_rmse': array([0.86554146, 0.86646693, 0.86578002, 0.86718664, 0.8665813 ]),
 'fit_time': (69.251051902771,
  69.39624381065369,
  69.03078699111938,
  65.1816177368164,
  65.33965349197388),
 'test_time': (23.455153226852417,
  23.963589668273926,
  21.785398483276367,
  22.28699827194214,
  22.248989820480347)}

In [62]:
print(sys.getsizeof(df) /1_000_000 , "MB")


646.501725 MB


In [45]:

# data = Dataset.load_from_df(
#     df[['Cust_Id', 'Movie_Id', 'Rating']][:1000000], reader)

# # # getting full dataset
# trainset = data.build_full_trainset()

# algo = SVD()

# algo.fit(trainset)


In [65]:
def recommend_movies(user_id, min_ratings_count=0, n_movies=10 , algo=svd):
    user = df_title.copy()
    user = user.reset_index()
    user = user[~user['Movie_Id'].isin(drop_movie_list)]
    # we remove from user table the movies that the user has already rated from get_prev_liked_movies function
    user = user[~user['Movie_Id'].isin(get_prev_liked_movies(user_id))]
    # print(user['Movie_Id'].isin(get_prev_liked_movies(user_id)).to_string(index=False))
    # print(get_prev_liked_movies(user_id))
    # print(user.head())
    user['Estimated rating'] = user['Movie_Id'].apply(
        lambda x: algo.predict(user_id, x).est)
    user.set_index('Movie_Id', inplace=True)
    
    # print(user.head())
    user = user.join(df_movie_summary)[['Name' , 'Estimated rating', 'count']]
    user = user[user['count'] > min_ratings_count]
    user = user.sort_values('Estimated rating', ascending=False)
    user.rename(columns={"Name": "movie name", "count": "number of ratings" } , inplace=True)
    # print(user.head(n_movies).to_string(index=False))
    print(user.head(n_movies).to_string(index=True))



In [67]:
recommend_movies(2342)


Liked movies by user 2342 are : 
                                             movie name  Estimated rating  number of ratings
Movie_Id                                                                                    
2114                                            Firefly          4.459413              10772
2102                             The Simpsons: Season 6          4.437537               8426
1476                           Six Feet Under: Season 4          4.423333              11829
2452      Lord of the Rings: The Fellowship of the Ring          4.357689             149866
270                          Sex and the City: Season 4          4.340023              36431
1495                                    Alias: Season 1          4.336051              16683
2162                                      CSI: Season 1          4.327326              17013
1795                                     MASH: Season 7          4.324654               5025
1692                                 

In [57]:
# for i in range(3 , 400):
#     print(algo.predict(i, 2452).est)
